# Title: **장단기메모리(Long Short Term Memory;LSTM)**를 이용한 **Sentimental Classification**
 
<img src="https://cfml.se/img/blog/sentiment_classification/top_img.png" width="600">

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb

import numpy as np

 - `tensorflow.keras.layers (layers)`: 딥러닝 네트워크를 설계할 때 층(layer) 관련 함수들(예:`Dense`, `Conv2D`, `MaxPooling2D`, `SimpleRNN`, `LSTM`)을 모아놓은 라이브러리

 - `tensorflow.keras.models.Model (Model)`: 생성한 층들을 연합하여 하나의 모델로 구성할 때 사용하는 함수

 - `tensorflow.keras.datasets`: TensorFlow에서 딥러닝 실습을 위해 제공해주는 데이터셋 (예: `mnist`, `cifar10`, `cifar100`, `imdb`)

 - `numpy (np)`: 다차원 데이터 처리를 위한 라이브러리 (참고: `pandas`-2차원 데이터에 특화된 라이브러리)

## Step 1-1) 데이터 불러오기

  <img src="https://miro.medium.com/max/1932/1*fNXlzk-u7VrDUdIASHqOIw.png" width="500">

 - 시계열 데이터의 경우, 데이터 유형(주식 가격과 같은 `숫자형 순차데이터`, `텍스트`, `음성`, `비디오`)  에 따라서 불러오는 방식과 전처리 하는 방식이 다릅니다.
 
 - imdb 데이터셋을 불러와 입력데이터(x)와 결과데이터(y)가 어떻게 구성되어 있는지 살펴봅시다.

 - imdb 데이터셋 불러오기

  - `num_words`: 단어의 빈도수가 높은 순서대로 숫자를 부여. `num_words=10000`이면 빈도수가 가장 높은 10000개의 단어만 불러오고, 나머지 단어들은 `oov (out-of-variable)` 형태로 불러옴

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [ ]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


 - 텍스트 데이터에 대해서는 `숫자형`으로 인코딩을 해주셔야 하는데

  - 1) 단어의 빈도수에 따라서 `사전(dictionary) **문자:숫자**`을 구성 

  - 2) 정의해 놓은 사전을 이용해서 숫자형으로 인코딩. 단, 모든 단어들을 전부 불러올 수는 없으니 빈도수에 따라서 가장 빈도수가 높은 N개의 단어들만 숫자로 변환

  - 3) 빈도수가 높은 N개에 포함되지 않는 단어들은 `OOV`로 표기

## Step 1-2) 데이터 전처리

 - RNN은 MLP와 다르게 임의의 길이의 시퀀스를 입력으로 받을 수 있으나 빠른 연산을 위해 **길이가 동일**한 시퀀스들을 입력으로 받습니다.
  
 - 모든 입력 문장의 길이를 동일하게 하기 위해 순차데이터 전처리 함수들을 모아놓은 `tf.keras.preprocessing.sequence` 패키지의 `pad_sequences()` 함수가 사용됩니다.

  - `pad_sequences()` 함수는 `문장의 길이를 얼마로 통일시킬지(maxlen)`를 입력으로 받아 
  
    - 지정된 길이보다 짧은 문장은 0을 추가해 길이를 맞추고 
    
    - 지정된 길이보다 긴 문장은 지정된 길이에 맞도록 자릅니다. 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_RNN = pad_sequences(sequences=x_train, maxlen=64, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=64, padding="pre", truncating="pre")

 - 텍스트 전처리가 제대로 되었는지 확인해봅시다.

In [ ]:
print(x_train_RNN.shape)
print(x_test_RNN.shape)

(25000, 64)
(25000, 64)


## Step 2) RNN 네트워크 구성

 - **layers** 모듈에 있는 **Input, SimpleRNN, Dense** 함수와 **Model** 함수를 이용하여 인공신경망을 설계해봅시다.

 - Input layer의 shape은 **sequence의 길이**로 설정합니다.

 - 각각의 단어를 one-hot encoding 해주었을 때 두 가지 문제가 발생합니다.

  1. **희소 (sparse)** 입력으로 인한 **학습 부진** 문제

      - 1 (one-hot)에 해당하는 가중치만 계산에 사용. 0에 해당하는 가중치들은 학습이 되지 않음.

  2. **Out of Memory (OOM)**, 메모리 부족 문제
      
 - 시계열 데이터 처리에서는 one-hot encoding (sparse encoding) 대신 dense embedding을 사용합니다 (**layers.Embedding()** 함수 이용)
  

In [ ]:
# seq_len: 시퀀스의 길이
# dim_embedding: 각각의 단어들을 인코딩해줄 때 인코딩 시키고 싶은 차원 
# dim_hidden: hidden state의 차원 (cell state의 차원은 hidden state의 차원과 동일)
# model_name: RNN을 쓸 것이냐, LSTM을 쓸 것이냐

def MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name):
    # 입력 계층
    review = layers.Input(shape=seq_len)

    # 임베딩 계층
    embedding = layers.Embedding(input_dim=num_words, output_dim=dim_embedding)(review)

    # 시계열 데이터 특징 추출 계층
    if model_name == "RNN":
      hidden = layers.SimpleRNN(units=dim_hidden, activation="tanh", return_sequences=False, return_state=False)(embedding)
    elif model_name == "LSTM":
      hidden = layers.LSTM(units=dim_hidden, activation="tanh", return_sequences=False, return_state=False)(embedding)

    # MLP를 이용한 최종 분류 계층
    prob = layers.Dense(units=1, activation="sigmoid")(hidden)

    return Model(inputs=review, outputs=prob)


  - **return_sequences**

    - `True`로 설정할 경우, 모든 cell의 hidden state들을 출력

  - **return_state**

    - `True`로 설정할 경우, 마지막 cell의 hidden state를 한 번 더 출력

## Step 3) 손실함수 (Loss), 모델 업데이트 알고리즘 (Optimizer), 평가지표 (metrics) 설정 (**compile**)

  - 분류 문제의 경우 손실함수(loss)로는 확률과 확률 사이의 차이를 측정하는데 특화된  `교차 엔트로피(Cross Entropy)`를 주로 사용합니다.
   - 이진 분류 (2개의 클래스 분류; 양성/음성, 개/고양이): `binary_crossentropy`
   - 다중 분류 (3개 이상의 클래스 분류; 개/고양이/사람):`categorical_crossentropy`
  
  - 모델 업데이트 알고리즘(optimizer)으로는 `경사하강법 (gradient descent)`에 기반한 `Adam`을 주로 사용합니다.
   - `tf.keras.optimizers` 라이브러리 안에서 다양한 종류의 optimizer 사용 가능

  - 평가지표(metrics)로는 `accuracy`를 사용하겠습니다.

## Step 4) 설계한 인공신경망 학습 (**fit**)

  - `batch_size`: parameter를 한 번 업데이트 할 때 몇 개의 데이터를 사용할 것인가

  - `epochs`: 학습데이터를 총 몇 번 복습시킬 것인가

  - `verbose`: 학습 경과를 보여줄 것인가

  - `validation_data`: 입력했을 경우, 1 epoch이 끝날 때 마다 validation_data에 대한 모델의 성능 출력

In [ ]:
num_words = 10000
seq_len = 64
dim_embedding = 256
dim_hidden = 128
model_name = "RNN"

# 모델 생성
model = MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name)

# 모델 출력
model.summary()

# 손실함수, 업데이트 알고리즘, 평가지표 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 입력 데이터 전처리
x_train_RNN = pad_sequences(sequences=x_train, maxlen=seq_len, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=seq_len, padding="pre", truncating="pre")

# 학습
model.fit(x_train_RNN, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_RNN, y_test))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64)]              0         
                                                                 
 embedding (Embedding)       (None, 64, 256)           2560000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               49280     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,609,409
Trainable params: 2,609,409
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 57s 69ms/step - loss: 0.5523 - accuracy: 0.7039 - val_loss: 0.4789 - val_accuracy: 0.7809
Epoch 2/5
782/782 [=====================

In [ ]:
num_words = 10000
seq_len = 64
dim_embedding = 256
dim_hidden = 128
model_name = "LSTM"

 모델 생성
model = MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name)

# 모델 출력
model.summary()

# 손실함수, 업데이트 알고리즘, 평가지표 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 입력 데이터 전처리
x_train_RNN = pad_sequences(sequences=x_train, maxlen=seq_len, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=seq_len, padding="pre", truncating="pre")

# 학습
model.fit(x_train_RNN, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_RNN, y_test))

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 64, 256)           2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,757,249
Trainable params: 2,757,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 16s 20ms/step - loss: 0.4382 - accuracy: 0.7916 - val_loss: 0.3907 - val_accuracy: 0.8307
Epoch 2/5
782/782 [==============================] - 16s 20ms/step - loss: 0.2818 - accuracy: 0.884

In [ ]:
num_words = 10000
seq_len = 512
dim_embedding = 256
dim_hidden = 128
model_name = "RNN"

# 모델 생성
model = MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name)

# 모델 출력
model.summary()

# 손실함수, 업데이트 알고리즘, 평가지표 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 입력 데이터 전처리
x_train_RNN = pad_sequences(sequences=x_train, maxlen=seq_len, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=seq_len, padding="pre", truncating="pre")

# 학습
model.fit(x_train_RNN, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_RNN, y_test))

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 512, 256)          2560000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 2,609,409
Trainable params: 2,609,409
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 196s 251ms/step - loss: 0.5730 - accuracy: 0.6886 - val_loss: 0.4773 - val_accuracy: 0.7922
Epoch 2/5
782/782 [==============================] - 197s 252ms/step - loss: 0.4556 - accuracy: 0

In [ ]:
num_words = 10000
seq_len = 512
dim_embedding = 256
dim_hidden = 128
model_name = "LSTM"

# 모델 생성
model = MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name)

# 모델 출력
model.summary()

# 손실함수, 업데이트 알고리즘, 평가지표 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 입력 데이터 전처리
x_train_RNN = pad_sequences(sequences=x_train, maxlen=seq_len, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=seq_len, padding="pre", truncating="pre")

# 학습
model.fit(x_train_RNN, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_RNN, y_test))


Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 512, 256)          2560000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,757,249
Trainable params: 2,757,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 42s 53ms/step - loss: 0.4570 - accuracy: 0.7802 - val_loss: 0.3740 - val_accuracy: 0.8388
Epoch 2/5
782/782 [==============================] - 42s 53ms/step - loss: 0.2566 - accuracy: 0.898

In [ ]:
num_words = 10000
seq_len = 1024
dim_embedding = 256
dim_hidden = 128
model_name = "RNN"

# 모델 생성
model = MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name)

# 모델 출력
model.summary()

# 손실함수, 업데이트 알고리즘, 평가지표 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 입력 데이터 전처리
x_train_RNN = pad_sequences(sequences=x_train, maxlen=seq_len, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=seq_len, padding="pre", truncating="pre")

# 학습
model.fit(x_train_RNN, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_RNN, y_test))


Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1024)]            0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1024, 256)         2560000   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 2,609,409
Trainable params: 2,609,409
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 427s 546ms/step - loss: 0.6543 - accuracy: 0.5954 - val_loss: 0.6191 - val_accuracy: 0.6354
Epoch 2/5
782/782 [==============================] - 421s 538ms/step - loss: 0.5338 - accuracy: 

In [ ]:
num_words = 10000
seq_len = 1024
dim_embedding = 256
dim_hidden = 128
model_name = "LSTM"

# 모델 생성
model = MyModel(num_words, seq_len, dim_embedding, dim_hidden, model_name)

# 모델 출력
model.summary()

# 손실함수, 업데이트 알고리즘, 평가지표 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 입력 데이터 전처리
x_train_RNN = pad_sequences(sequences=x_train, maxlen=seq_len, padding="pre", truncating="pre")
x_test_RNN = pad_sequences(sequences=x_test, maxlen=seq_len, padding="pre", truncating="pre")

# 학습
model.fit(x_train_RNN, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_RNN, y_test))


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1024)]            0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 1024, 256)         2560000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 2,757,249
Trainable params: 2,757,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 70s 90ms/step - loss: 0.4621 - accuracy: 0.7781 - val_loss: 0.3309 - val_accuracy: 0.8652
Epoch 2/5
782/782 [==============================] - 69s 88ms/step - loss: 0.2708 - accuracy: 0.89